In [1]:
from os.path import join, exists
from os import makedirs
import re

In [2]:
# source taxonomy to match against reference
source_fp = '/Users/nbokulich/Desktop/projects/mockrobiota/data/mock-11/source/taxonomy.tsv'

# reference taxonomy to use for annotation
ref_taxa_fp = '/Users/nbokulich/Desktop/ref_dbs/SILVA123_QIIME_release/taxonomy/18S_only/99/taxonomy_7_levels.txt'

# destination directory path
destination_dir = '/Users/nbokulich/Desktop/projects/mockrobiota/data/mock-11/silva/123/99_otus/'

In [13]:
print(l)
print(name)
print(taxon)
l.strip().split('\t')[0]

Eukaryota;Metazoa;Chordata;Craniata;Vertebrata;Euteleostomi;Mammalia;Eutheria;Euarchontoglires;Primates;Haplorrhini;Catarrhini;Hominidae;Homo;Homo sapiens	0.0833333334

Eukaryota;Metazoa;Chordata;Craniata;Vertebrata;Euteleostomi;Mammalia;Eutheria;Euarchontoglires;Primates;Haplorrhini;Catarrhini;Hominidae;Homo;Homo sapiens
['Homo', 'sapiens']


'Eukaryota;Metazoa;Chordata;Craniata;Vertebrata;Euteleostomi;Mammalia;Eutheria;Euarchontoglires;Primates;Haplorrhini;Catarrhini;Hominidae;Homo;Homo sapiens'

In [3]:
# generate dict of {name: (genus, species, abundances)}
with open(source_fp, "r") as source:
    sample_list = source.readline().strip().split('\t')[1:]
    taxa = {}
    for l in source:
        # convert abundances to float
        abundances = list(map(float, l.strip().split('\t')[1:]))
        name = l.strip().split('\t')[0]
        taxon = re.split(' |_', name.split(';')[-1])[0:2]
        taxa[name] = (taxon, abundances)

ValueError: could not convert string to float: ';Trichophyton_sp'

In [7]:
taxa

{'Eukaryota;Fungi;Ascomycota;Eurotiomycetes;Onygenales;Arthrodermataceae;Trichophyton': (['Trichophyton'],
  [0.0833333333]),
 'Eukaryota;Fungi;Ascomycota;Eurotiomycetes;Onygenales;Onygenaceae;Coccidioides;Coccidioides_immitis': (['Coccidioides',
   'immitis'],
  [0.0833333333]),
 'Eukaryota;Fungi;Ascomycota;Saccharomycetes;Saccharomycetales;Incertae_sedis;Candida;Candida_albicans': (['Candida',
   'albicans'],
  [0.0833333333]),
 'Eukaryota;Fungi;Ascomycota;Saccharomycetes;Saccharomycetales;Incertae_sedis;Candida;Candida_tropicalis': (['Candida',
   'tropicalis'],
  [0.0833333333]),
 'Eukaryota;Fungi;Ascomycota;Saccharomycetes;Saccharomycetales;Saccharomycetaceae;Candida;Candida_lusitaniae': (['Candida',
   'lusitaniae'],
  [0.0833333333]),
 'Eukaryota;Fungi;Ascomycota;Saccharomycetes;Saccharomycetales;Saccharomycetaceae;Saccharomyces;Saccharomyces_cerevisiae': (['Saccharomyces',
   'cerevisiae'],
  [0.0833333333]),
 'Eukaryota;Fungi;Ascomycota;Schizosaccharomycetes;Schizosaccharomyce

In [180]:
# parse ref taxonomy
with open(ref_taxa_fp, "r") as ref:
    ref_taxa = {l.strip().split('\t')[1]: (l.strip().split('\t')[1].split(';')[-2],
                                           l.strip().split('\t')[1].split(';')[-1],
                                           l.strip().split('\t')[0]
                                          ) for l in ref}


In [242]:
def add_lists(l1, l2):
    newlist = [sum(tup) for tup in zip(l1, l2)]
    return newlist


In [243]:
a1 = [0.1, 0.2, 0.3]
a2 = [0.1, 0.2, 0.3]
a3 = add_lists(a1, a2)
a3

[0.2, 0.4, 0.6]

In [244]:
# find matching taxonomies
species_match = 0
genus_match = 0
no_match = 0
count = len(taxa)

duplicates = []
seq_ids = dict()
new_taxa = dict()

for name, t in taxa.items():
    match = 'None'
    
    # search for match at genus, then species level
    for full, partial in ref_taxa.items():
        if t[0][0] in partial[0]:
            if t[0][1] in partial[1]:
                match = 'species'
                seq_ids[full] = partial[2]
                break
            else:
                match = 'genus'
                genus = ';'.join(full.split(';')[:-1])
                
    # now add match to new_taxa
    if match == 'species':
        species_match += 1
        if full not in new_taxa.keys():
            new_taxa[full] = (name, t[1])
        else:
            # if species is replicated, collapse abundances
            
            new_taxa[full] = (name, add_lists(new_taxa[full][1], t[1]))
            #new_taxa[full] = (name, [sum(tup) for tup in zip(new_taxa[full][1], t[1])])
            duplicates.append(full)

    elif match == 'genus':
        genus_match += 1
        if genus not in new_taxa.keys():
            new_taxa[genus] = (name, t[1])
        else:
            # if genus is replicated, collapse abundances
            new_taxa[genus] = (name, add_lists(new_taxa[genus][1], t[1]))
            #new_taxa[genus] = (name, [sum(tup) for tup in zip(new_taxa[genus][1], t[1])])
            duplicates.append(genus)

    # if failed, user needs to manually search and input new string
    else:
        no_match += 1
        print('{0} has no matches to {1}.'.format(name, ref_taxa_fp))
        print('Perform a manual search of your reference database to match the')
        print('nearest basal lineage. A good place to start is NCBI taxonomy')
        print('(https://www.ncbi.nlm.nih.gov/taxonomy) to find the current')
        print('accepted taxonomy, then use grep or similar to search for the')
        print('nearest lineage in your reference. Assign the nearest basal lineage.')
        print('For example, if Lactobacillus casei has no species or genus-level')
        print('matches in the database, but Lactobacillales is present, assign')
        print('the taxonomy string up to Lactobacillales.')
        print('\nEnter the correct taxonomy for the basal lineage here:')
        lineage = 'fake'#input('> ')
        #failures.append(name)
        #new_taxa[name] = (name, t[1])
        if lineage not in new_taxa.keys():
            new_taxa[lineage] = (name, t[1])
        else:
            # if genus is replicated, collapse abundances
            new_taxa[lineage] = (name, add_lists(new_taxa[lineage][1], t[1]))
            #new_taxa[lineage] = (name, [sum(tup) for tup in zip(new_taxa[lineage][1], t[1])])
            duplicates.append(lineage)


Howardella ureilytica DSM 15118 has no matches to /Users/nbokulich/Desktop/ref_dbs/gg_13_8_otus/taxonomy/99_otu_taxonomy.txt.
Perform a manual search of your reference database to match the
nearest basal lineage. A good place to start is NCBI taxonomy
(https://www.ncbi.nlm.nih.gov/taxonomy) to find the current
accepted taxonomy, then use grep or similar to search for the
nearest lineage in your reference. Assign the nearest basal lineage.
For example, if Lactobacillus casei has no species or genus-level
matches in the database, but Lactobacillales is present, assign
the taxonomy string up to Lactobacillales.

Enter the correct taxonomy for the basal lineage here:


In [245]:
new_taxa

{'fake': ('Howardella ureilytica DSM 15118', [0.037037037]),
 'k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__Bacteroidaceae; g__Bacteroides': ('Bacteroides cellulosilyticus DSM 14838',
  [0.148148148]),
 'k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__Bacteroidaceae; g__Bacteroides; s__eggerthii': ('Bacteroides eggerthii BEI HM-210',
  [0.037037037]),
 'k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__Bacteroidaceae; g__Bacteroides; s__fragilis': ('Bacteroides fragilis ATCC 23745',
  [0.037037037]),
 'k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__Bacteroidaceae; g__Bacteroides; s__ovatus': ('Bacteroides ovatus DSM 1896',
  [0.037037037]),
 'k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__Bacteroidaceae; g__Bacteroides; s__uniformis': ('Bacteroides uniformis DSM 6597',
  [0.037037037]),
 'k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__Porphyromonadaceae; g__Paraba

In [236]:
print('{0} species-level matches ({1:.1f}%)'.format(species_match, species_match/count*100))
print('{0} genus-level matches ({1:.1f}%)'.format(genus_match, genus_match/count*100))
if no_match > 0:
    print('{0} FAILURES ({1:.1f}%)'.format(no_match, no_match/count*100))
    
if len(duplicates) > 0:
    print('\n{0} duplicates:'.format(len(duplicates)))
    for dup in duplicates:
        print(dup)



13 species-level matches (48.1%)
13 genus-level matches (48.1%)
1 FAILURES (3.7%)

6 duplicates:
k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__Bacteroidaceae; g__Bacteroides
Parabacteroides distasonis JCM 13401
k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__Bacteroidaceae; g__Bacteroides
k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__Bacteroidaceae; g__Bacteroides
k__Bacteria; p__Firmicutes; c__Clostridia; o__Clostridiales; f__Peptostreptococcaceae; g__[Clostridium]
k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__Porphyromonadaceae; g__Parabacteroides


## Write to file

In [208]:
if not exists(destination_dir):
    makedirs(destination_dir)
    
with open(join(destination_dir, 'expected-taxonomy.tsv'), "w") as dest:
    dest.write('Taxonomy\t{0}\n'.format('\t'.join(sample_list)))
    for name, t in new_taxa.items():
        abundances = ["{:.10f}".format(n) for n in t[1]]
        dest.write('{0}\t{1}\n'.format(name, '\t'.join(abundances)))

with open(join(destination_dir, 'database-identifiers.tsv'), "w") as dest:
    for t, seq_id in seq_ids.items():
        dest.write('{0}\t{1}\n'.format(t, seq_id))
